In [113]:
#!/usr/bin/env python3

import os
#from keras.datasets import mnist
import tensorflow as tf
#import numpy as np
from PIL import Image
import os, sys
import glob

from tensorflow import keras
#import keras
#from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras import backend as K

from keras.utils import to_categorical
import numpy as np
from keras.models import load_model

import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

#Python --version: Python 3.7 :: Anaconda, Inc.
'-------------------------------------------------------'
'               EXTERNAL PATHS                          '
'-------------------------------------------------------'
'''path of external device '''
rootDir = "/Volumes/Disk E/"
datasDir = rootDir + "/crohme2016_inkml_datasets/CROHME2016_data/"

imagesDir = rootDir + "/crohme2016_inkml_images/"
'''now get train valid test dir'''

trainDir = datasDir + 'Task-2-Symbols/task2-trainSymb2014/'
trainDirDatasets = trainDir + 'trainingSymbols/'
trainDir_ground_truth = trainDir + 'trainingSymbols/iso_GT.txt'
trainDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/train_images"

testDir = datasDir + 'Task-2-Symbols/task2-testSymbols2014/'
testDir_ground_truth = testDir + "testSymbols_2016_iso_GT.txt"
testDirDatasets = testDir + "testSymbols/"
testDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/test_images"

valDir = datasDir + 'Task-2-Symbols/task2-validation-isolatedTest2013b/'
valDirDatasets = datasDir + 'Task-2-Symbols/task2-validation-isolatedTest2013b/'
valDir_ground_truth = valDirDatasets + 'iso_GT.txt'
valDatasImmagesDir = "/Volumes/Disk E/crohme2016_inkml_images/val_images"

models_save = "/Volumes/Disk E/models_save/"


In [114]:
class CNN(object):

    differents_characters = 101
    image_len = 28
    X_train_len = 85802
    X_test_len = 18435
    X_val_len = 12504

    def __init__(self):
        self.X_train, self.X_test, self.X_val = [], [], []
        self.y_train, self.y_test, self.y_val = [], [], []

        self.y_train_ordered_labels = []
        self.y_test_ordered_labels = []
        self.y_val_ordered_labels = []
        
        
       
        #self.gt = [ {sortedDictTrain}  , {sortedDictTest}, {sortedDictval} ]
        #sortedDictTrain = ')' : [chemin de ttes les images de parenthèses], '9': [chemin de ttes les images de 9],  ...
        self.gt = {} 

        '''first load grounth truth and prevent junk data to load '''
        self.get_all_images_categorical()
        '''second load images and prevent junk images  to load '''
        self.get_all_images_Matrix()

        self.X_train, self.X_test, self.X_val = np.array(self.X_train), np.array(self.X_test), np.array(self.X_val)
        self.y_train, self.y_test, self.y_val = np.array(self.y_train), np.array(self.y_test), np.array(self.y_val)
        
        self.X_train, self.X_test, self.X_val= (self.X_train - np.mean(self.X_train))/np.std(self.X_train), (self.X_test - np.mean(self.X_test))/np.std(self.X_test) , (self.X_val - np.mean(self.X_val))/np.std(self.X_val)
        
        'normalize R G B: L=X_train R, A=X_train G, B=X_train B'
        # L, A, B = imgLABs_arr[ :, :, :, 0 : 1 ], imgLABs_arr[ :, :, :, 1 : 2 ], imgLABs_arr[ :, :, :, 2 : 3 ]
        #L_mean, L_std = np.mean( L ), np.std( L )
        #A_mean, A_std = np.mean( A ), np.std( A )
        #B_mean, B_std = np.mean( B ), np.std( B )
        #L, A, B = ( L - L_mean ) / L_std, ( A - A_mean ) / A_std, ( B - B_mean ) / B_std
        #AB = np.concatenate( ( A, B ), axis = 3)

    def get_matrix_from_image(self, infilename):
        img = Image.open(infilename)
        img.load()
        imageMatrix = np.asarray(img, dtype="float32")/255.0
        return imageMatrix
    #------------------------------------------------------------------------------------------------------#
    #                         GET TRAIN RESAUL
    #-------------------------------------------------------------------------------------------------------#
    def get_which_character(y):
        pass
    #------------------------------------------------------------------------------------------------------#
    # result: X_train = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso85802.png ]
    #         X_test = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso18402.png  ]
    #         X_val = [imageMatrix_iso0.png, imageMatrix_iso0.png, ........., imageMatrix_iso18402.png ]
    #--------------------------------------------------------------------------------------------------------#
    def get_all_images_Matrix(self):
        directories = [trainDatasImmagesDir, testDatasImmagesDir, valDatasImmagesDir]
        if not self.gt:
            print("Error no ground truth to prevent to load junk image")
            return
        for index, data_dir_abs_path in enumerate(directories):
            print("loding image ..............\n")
            if os.path.isdir(data_dir_abs_path):
                if index == 0:
                    slice_start, slice_stop = 3, -4
                    ground_truth = self.gt['train_gt']
                if index == 1:
                    slice_start, slice_stop = 3, -4
                    ground_truth = self.gt['test_gt']
                if index == 2:
                    slice_start, slice_stop = 4, -4
                    ground_truth = self.gt['val_gt']

                print(".... start loading  in folder: {}".format(data_dir_abs_path))
                #get array of files
                images_files_array = sorted(glob.glob(data_dir_abs_path + os.sep + "*.png"))
               
                #sort files array
                images_files_array = sorted(images_files_array, key=lambda name: int(
                (os.path.basename(name))[slice_start: slice_stop]))
                
                #inkml_files = inkml_files.sort(key=self.sortGlobFilesArray)
                for j, image_file_abs_path in enumerate(images_files_array):
                    
                    file_name_index = image_file_abs_path.split('/')[-1]
                    file_name_index = file_name_index[int(slice_start): int(slice_stop)]
                    #print(".............fle check: {} .............".format(file_name_index))
                    file_is_junk = True
                    for key in ground_truth:
                        if file_name_index in ground_truth[key]:
                            file_is_junk = False
                    if  False == file_is_junk:
                        imageMatrix = self.get_matrix_from_image(image_file_abs_path).reshape(28, 28, 1)
               
                        if index == 0:
                            self.X_train.append(imageMatrix)
                        if index == 1:
                            self.X_test.append(imageMatrix)
                        if index == 2:
                            self.X_val.append(imageMatrix)
                    #else: 
                        #print('...............Found JUNK FILE .....................\n')
                if index == 0:
                    print("------- LEN IMAGES FILES: ", len(self.X_train))
                if index == 1:
                    print("------- LEN IMAGES FILES: ", len(self.X_test))
                if index == 2:
                    print("------- LEN IMAGES FILES: ", len(self.X_val))

            else:
                print("Error: External Devices not found\n")
        
        
    #---------------------------------------------------------------------------------------------------#
    #  Result: y_train = Matrix 101 x 101, y_test = Matrix 101 x 101, y_val = Matrix 101 x 101
    #---------------------------------------------------------------------------------------------------#
    def get_all_images_categorical(self):
        gt_directories_files = [trainDir_ground_truth, testDir_ground_truth, valDir_ground_truth]
        imagesDir = [trainDatasImmagesDir, testDatasImmagesDir, valDatasImmagesDir]
        
        for index, abs_gt_file_path in enumerate(gt_directories_files):
            #print("Start reading grount truth ..... {}" .format(os.path.basename(abs_gt_file_path)))
            sorted_gt, total_character, y = self.read_gt(abs_gt_file_path, imagesDir[index])
            
            length = len(sorted_gt.keys())
            
            for i, el in enumerate(y):
                x = np.zeros(length, dtype=np.float32)
                x[el] = 1.0
                        
                if index == 0:
                    self.y_train.append(x)
                    self.gt['train_gt'] = sorted_gt
                    self.y_train_ordered_labels = sorted_gt.keys()
                if index == 1:
                    self.y_test.append(x)
                    self.gt['test_gt'] = sorted_gt
                    self.y_test_ordered_labels = sorted_gt.keys()
                if index == 2:
                    self.y_val.append(x)
                    self.gt['val_gt'] = sorted_gt
                    self.y_val_ordered_labels = sorted_gt.keys()
    
    
    #------------------------------------------------------------------------------#
    # return a Dictionary
    # {
    # ')' : [abs_path_image1, abs_path_imag2, ...]
    #    .....................................
    #   '!'  : [abs_path_image14, abs_path_imag267, ...]
    # }
    #------------------------------------------------------------------------------#
    def read_gt(self, abs_gt_file_path, imagesDir):
        '''Loads the routes of the png files'''
        try:
            fileName = os.path.basename(abs_gt_file_path)
            print("reading ground truth {} ..................".format(fileName))
            with open(abs_gt_file_path) as file:
                lines = file.readlines()

                aDict = {}
                aLabelsSet = []
                total_character = 0
                all_labels = []
                
                for index, line in enumerate(lines):
                    line = line.strip()

                    parts = line.split(',')
                    label = parts[1].strip()

                    data = parts[0].split('_')
                    index = data[-1].strip()
                    if(label != 'junk'):
                        all_labels.append(label)
                    if label not in aLabelsSet and label != 'junk':
                        aLabelsSet.append(label)

                    if label in aDict and label != 'junk':
                        #aDict[label].append(imagesDir + '/' + 'iso' + index + '.png')
                        aDict[label].append(index)
                    else:
                        #aDict[label] = [imagesDir + '/' + 'iso' + index + '.png']
                        aDict[label] = [index]
                    total_character = total_character + 1

            #sort dictionary but others method doesn't work
            bDict = {}
            y = []
            for label in aLabelsSet:
                bDict[label] = aDict[label]
            for label in all_labels:
                for i, el in enumerate(aLabelsSet):
                    if(el == label):
                        y.append(i)

            #aDict = OrderedDict(sorted(aDict.items(), key=lambda t: t[0]))
            # print("dictionnary sorted keys : ", bDict.keys())
            print("Finish readin ground {} truth: {}".format(fileName, len(bDict)))
            #print("bDict length:........", len(bDict))
            return bDict, total_character,y

        except FileNotFoundError as e:
            print(e)
    
    def create_model(self):
        model = keras.Sequential()
        model.add(InputLayer(input_shape = (28, 28, 1)  ))
        #adding layers
        model.add(Conv2D(64, (3,3), padding='same', activation='relu', strides=2, kernel_initializer='truncated_normal'))
        model.add(MaxPooling2D( (3, 3), strides = 1, padding='same' ) )
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2,
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=1,
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2,
                 kernel_initializer='truncated_normal'))
        model.add(MaxPooling2D( (3, 3), strides = 1, padding='same' ) )
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=1,
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=1,
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=1,
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=1,
                 kernel_initializer='truncated_normal'))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same',
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same',
                 kernel_initializer='truncated_normal'))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(32, (3, 3), activation=None, padding='same',
                 kernel_initializer='truncated_normal'))
        model.add(Conv2D(2, (3, 3), activation=None, padding='same',
                 kernel_initializer='truncated_normal'))
        model.add(UpSampling2D((2, 2)))
        
        return model

    def compile_model(self, model):
        #compile model using accuracy to measure model performance
        # Compile the model
        #model.compile(
            #loss=keras.losses.mean_squared_error, optimizer='sgd')
        optimizer = tf.keras.optimizers.RMSprop( lr = 0.0005, decay = 1e-5 )
        model.compile( optimizer=optimizer, loss='mse', metrics=['acc'] )
        #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def train_model(self, model):
        #fit the model
        #model.fit(self.X_train, self.y_train, batch_size=500, epochs=5)
        
        #model.fit(self.X_train, self.y_train, validation_data=(self.X_test, self.y_test), epochs=5,batch_size=500)
        # Evaluate the model on test set
        
        #score=model.evaluate(self.X_test, self.y_test, verbose=0)
        
        # Print test accuracy
        #print('\nTest accuracy: {}'.format(score[1]*100))
        model.fit( x=self.X_train, y=self.y_train, validation_data=(self.X_test, self.y_test), batch_size=1, epochs=1800)
        return model
    
    def save_model(self, model):
        # save model and architecture to single file
        model.save(models_save + "model.h5")
        print("Saved model to disk")
        return model
    
    def load_saved_model(self, model):
        model = load_model(models_save + "model.h5")
        # summarize model.
        model.summary()
        return model

    def evaluate_model(self, model):
        score = model.evaluate(self.X_val, self.y_val, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

    def model_predict(self, model, start):
        #predict first 4 images in the test set
        return model.predict(self.X_val[start : start + 1])
        
                
        


In [115]:
cnn = CNN()

reading ground truth iso_GT.txt ..................
Finish readin ground iso_GT.txt truth: 101
reading ground truth testSymbols_2016_iso_GT.txt ..................
Finish readin ground testSymbols_2016_iso_GT.txt truth: 101
reading ground truth iso_GT.txt ..................
Finish readin ground iso_GT.txt truth: 101
loding image ..............

.... start loading  in folder: /Volumes/Disk E/crohme2016_inkml_images/train_images
------- LEN IMAGES FILES:  85802
loding image ..............

.... start loading  in folder: /Volumes/Disk E/crohme2016_inkml_images/test_images
------- LEN IMAGES FILES:  10019
loding image ..............

.... start loading  in folder: /Volumes/Disk E/crohme2016_inkml_images/val_images
------- LEN IMAGES FILES:  6083


In [116]:
cnn.y_train_ordered_labels

dict_keys(['(', '-', ')', 'n', '=', 'i', '\\sum', 'S', '1', 'r', '\\theta', '2', '\\pi', '8', 'd', 'a', '4', 'c', 'b', '+', '3', '7', 'y', '0', 'x', '\\sqrt', 'k', '\\sin', '\\cos', '\\pm', 'p', 'u', 'q', 'P', 'j', '\\leq', '|', '\\lt', '!', '\\ldots', 'f', 'C', 'B', 'A', 'R', 'm', 'COMMA', 't', '\\prime', 'w', 's', '\\int', '\\alpha', 'H', '\\Delta', 'I', '\\in', 'M', 'X', '\\sigma', 'T', 'E', 'L', '\\forall', 'V', '\\mu', '\\{', '\\}', 'N', '\\neq', 'G', 'e', 'F', 'g', '\\phi', '\\exists', '\\gt', 'z', '\\log', '9', '\\div', '5', '\\times', 'o', 'v', '6', '/', '\\gamma', '\\beta', 'h', '\\geq', 'Y', '\\infty', 'l', '.', '\\rightarrow', '\\lim', '[', ']', '\\tan', '\\lambda'])

In [117]:
cnn.y_test_ordered_labels

dict_keys(['x', 'k', '+', 'y', '4', '8', '\\sqrt', '2', '6', 'q', 't', '=', '-', 'p', 'e', '1', ')', '(', 'd', '0', 'm', '\\geq', 'u', '\\theta', '3', '\\neq', 'n', 'C', '\\ldots', '\\gamma', '\\div', '5', '7', '9', 'COMMA', '\\int', 'g', '\\rightarrow', '\\infty', '\\lim', 'a', 'b', '\\sin', 'L', '\\leq', 'c', '\\cos', 'o', 'R', 'r', '\\beta', '.', 'S', 'I', 'X', '\\pi', '\\mu', 'v', '\\times', 'Y', '/', 'f', 'z', '\\sum', '\\alpha', 'F', '\\log', '\\prime', 'T', 'i', 'B', 'l', ']', '[', '\\exists', 'w', 'H', 'A', 'M', '\\pm', 'N', 'V', 'P', 'j', 'E', '|', '\\lt', '\\tan', '\\gt', '\\phi', 's', '\\Delta', '!', '\\{', '\\}', '\\sigma', 'G', '\\forall', '\\in', '\\lambda', 'h'])

In [118]:
cnn.y_val_ordered_labels

dict_keys(['a', 'n', '1', '+', '-', 'X', '\\ldots', '=', 'P', '0', 'c', 'v', '(', ')', 'e', 'w', 'k', 'COMMA', 'x', 'd', 'p', 'T', '\\Delta', 'G', 'z', 'R', 'g', 'A', '.', 'm', 'q', 'u', 's', '\\pi', '3', '6', '2', 'N', 'i', 'E', 'B', 'S', '\\times', 'F', 'f', 'r', 'M', 't', '\\prime', 'y', '/', 'b', '\\gt', 'l', '\\mu', 'L', 'j', 'H', '\\infty', '\\int', '\\sqrt', '|', 'C', '\\lt', '4', '5', '\\leq', 'I', '\\alpha', '\\gamma', '\\sigma', '8', '\\in', 'o', '\\{', '\\}', ']', '[', '\\beta', 'V', '7', '\\theta', '\\log', '\\geq', 'h', '\\cos', '\\neq', '\\sin', '\\sum', '\\tan', '\\phi', '\\rightarrow', '\\lim', 'Y', '9', '\\pm', '!', '\\exists', '\\forall', '\\lambda', '\\div'])

In [121]:
model = cnn.create_model()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [120]:
model = cnn.compile_model(model)

In [111]:
model = cnn.train_model(model)

Train on 85802 samples, validate on 10019 samples
Epoch 1/1800
 1601/85802 [..............................] - ETA: 9:24 - loss: 1.2907 - accuracy: 0.6615

KeyboardInterrupt: 

In [81]:
cnn.model_predict(model ,0)

TypeError: model_predict() takes 2 positional arguments but 3 were given

In [82]:
model.predict(cnn.X_val[0:1])

array([[1.66862635e-10, 9.95224703e-09, 1.38151590e-09, 1.50686334e-04,
        3.41216388e-09, 1.10437199e-07, 1.39720173e-08, 1.18959204e-10,
        7.91695318e-04, 5.77257606e-06, 1.33835567e-06, 5.60225279e-04,
        4.31640814e-07, 1.05810315e-07, 2.04762764e-05, 9.84200776e-01,
        1.29356061e-03, 1.24378012e-05, 5.51517099e-09, 1.60859895e-06,
        4.62428972e-07, 1.08008644e-05, 3.21167227e-06, 2.53103872e-06,
        5.15664369e-03, 1.46104240e-10, 8.30289082e-06, 1.16428043e-06,
        6.07252014e-06, 1.10353762e-08, 6.74697409e-09, 4.50776797e-03,
        7.88829402e-06, 9.07459707e-09, 4.66488936e-09, 6.58876047e-08,
        1.94051353e-09, 2.55968331e-07, 1.70574932e-09, 1.52992081e-11,
        9.52555812e-10, 1.86429361e-07, 1.39081786e-08, 5.09674874e-06,
        5.57849307e-06, 2.13248882e-06, 4.70390980e-08, 4.27921691e-07,
        3.07570467e-08, 5.52212036e-08, 1.25047023e-07, 8.00979616e-10,
        2.94296653e-03, 7.83644438e-08, 6.18273987e-07, 6.495633